In [1]:
pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.3 MB/s eta 0:00:00


In [2]:
pip install transformers torch elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 4.1 MB/s eta 0:00:00


In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    "https://my-elasticsearch-project-a610a8.es.us-west-2.aws.elastic.cloud:443",
    api_key="TzRYd09wVUJzNHJ1TnVZQ2VTQWE6dU5TUmhEaGZDX2xTa0NSVjQteWhydw==")

In [ ]:
from elasticsearch import Elasticsearch
from transformers import AutoTokenizer, AutoModel, pipeline
import random
import torch
import numpy as np
# Load pre-trained model and tokenizer from Hugging Face
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Zero-shot classifier from Hugging Face for user classification based on transactions
classifier = pipeline('zero-shot-classification')

# Define the index mapping for user transactions with vector field
index_name = 'user_transactions'
mapping = {
    "mappings": {
        "properties": {
            "user_id": {"type": "keyword"},
            "transaction_embedding": {"type": "dense_vector", "dims": 384},  # Dimension should match model output
            "transaction_text": {"type": "text"}
        }
    }
}

# Create the index if it doesn't exist
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")


def get_transaction_embedding(transaction_text):
    """Generate embeddings for transaction descriptions using a Hugging Face model."""
    inputs = tokenizer(transaction_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Take the mean of token embeddings
    return embedding


# Sample transaction categories and text templates
categories = ["Electronics", "Groceries", "Clothing", "Entertainment", "Books", "Technology", "Home Decor", "Fitness"]
sample_items = [
    "Laptop", "Smartphone", "TV", "Refrigerator", "Washing Machine", "Shoes", "T-shirt", "Washing Powder",
    "Pants", "Sofa", "Headphones", "Fitness Equipment", "Books", "Magazine", "Camera", "Couch", "Microwave",
    "Gaming Console", "Portable Speaker", "Tablet", "Vacuum Cleaner"
]

def generate_transaction(user_id, num_transactions=20):
    """Generate random transactions for a user and store them in Elasticsearch."""
    for i in range(num_transactions):
        # Randomly generate a transaction description and category
        item = random.choice(sample_items)
        category = random.choice(categories)
        transaction_text = f"Bought {item} from the {category} section."

        # Generate the embedding for the transaction description
        embedding = get_transaction_embedding(transaction_text)

        # Prepare the document to index
        doc = {
            "user_id": user_id,
            "transaction_embedding": embedding.flatten().tolist(),  # Convert numpy array to list
            "transaction_text": transaction_text
        }

        # Index the document in Elasticsearch
        es.index(index=index_name, document=doc)
        print(f"Transaction {i+1} indexed for user {user_id}: {transaction_text}")


# Example: Generate and index 20 transactions for user 'user_123'
generate_transaction("user_123", num_transactions=20)


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Index 'user_transactions' already exists.
Transaction 1 indexed for user user_123: Bought Fitness Equipment from the Groceries section.
Transaction 2 indexed for user user_123: Bought Books from the Electronics section.
Transaction 3 indexed for user user_123: Bought Couch from the Entertainment section.
Transaction 4 indexed for user user_123: Bought Camera from the Electronics section.
Transaction 5 indexed for user user_123: Bought Washing Powder from the Fitness section.
Transaction 6 indexed for user user_123: Bought Portable Speaker from the Books section.
Transaction 7 indexed for user user_123: Bought Pants from the Technology section.
Transaction 8 indexed for user user_123: Bought Tablet from the Clothing section.
Transaction 9 indexed for user user_123: Bought Tablet from the Groceries section.
Transaction 10 indexed for user user_123: Bought Fitness Equipment from the Clothing section.
Transaction 11 indexed for user user_123: Bought Microwave from the Groceries section.
Tr

In [ ]:
# Function to classify user based on their transaction history
def classify_user(user_id):
    """Classify user based on their transaction categories."""
    query = {
        "query": {
            "term": {
                "user_id": user_id
            }
        }
    }

    # Search for the user's transactions in Elasticsearch
    results = es.search(index=index_name, body=query)

    # Concatenate transaction categories to form a classification input
    transaction_categories = [hit["_source"]["transaction_text"].split('from the')[1].split('section')[0] for hit in results['hits']['hits']]

    # Perform classification using Hugging Face model
    classification = classifier(" ".join(transaction_categories), candidate_labels=["Technology", "Sports", "Finance", "Entertainment", "Health"])

    return classification


# Example: Classify user 'user_123'
user_classification = classify_user("user_123")
print(f"User classification: {user_classification}")

User classification: {'sequence': ' Electronics   Fitness   Fitness   Electronics   Groceries   Entertainment   Electronics   Books   Clothing   Fitness ', 'labels': ['Entertainment', 'Technology', 'Health', 'Sports', 'Finance'], 'scores': [0.5165974497795105, 0.39473316073417664, 0.0738568976521492, 0.011545686051249504, 0.0032667291816323996]}


In [ ]:
# Function to recommend news based on the user's classification
def recommend_news(user_classification):
    """Recommend news sources based on the user's classification."""
    category = user_classification['labels'][0]  # Top category predicted by the model
    news_recommendations = {
        "Technology": ["TechCrunch", "Wired", "The Verge"],
        "Sports": ["ESPN", "Bleacher Report", "Sports Illustrated"],
        "Finance": ["The Wall Street Journal", "Forbes", "Bloomberg"],
        "Entertainment": ["Variety", "The Hollywood Reporter", "Buzzfeed"],
        "Health": ["WebMD", "Healthline", "Medical News Today"]
    }

    return news_recommendations.get(category, ["General News"])

# Example: Recommend news based on user classification
recommended_news = recommend_news(user_classification)
print(f"Recommended news sources for user: {recommended_news}")

Recommended news sources for user: ['Variety', 'The Hollywood Reporter', 'Buzzfeed']


In [ ]:
from elasticsearch import Elasticsearch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import random
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define index names
transactions_index = 'transactions_index'
profiles_index = 'profiles_index'  # Index for storing user profiles
profile_knowledge_base_index = 'profile_knowledge_base_index'  # Knowledge base index for profile categories
news_genre_index = 'news_genre_index'  # Index for storing news genres and their embeddings

# Define the index settings and mappings for creating the transactions index
index_body = {
    "mappings": {
        "properties": {
            "user_id": {"type": "keyword"},
            "transaction_text": {"type": "text"},
            "transaction_embedding": {
                "type": "dense_vector",
                "dims": 768  # Ensure that this matches the embedding dimensions of your model
            }
        }
    }
}

# Create the transactions index if it doesn't already exist
if not es.indices.exists(index=transactions_index):
    es.indices.create(index=transactions_index, body=index_body)
    print(f"Index '{transactions_index}' created successfully!")
else:
    print(f"Index '{transactions_index}' already exists.")

# Create profiles index with vector field for embeddings
profile_index_body = {
    "mappings": {
        "properties": {
            "category_name": {"type": "keyword"},
            "category_embedding": {
                "type": "dense_vector",
                "dims": 768  # Embedding size for profile categories
            }
        }
    }
}

if not es.indices.exists(index=profiles_index):
    es.indices.create(index=profiles_index, body=profile_index_body)
    print(f"Index '{profiles_index}' created successfully!")
else:
    print(f"Index '{profiles_index}' already exists.")

# Create profile knowledge base index
profile_knowledge_base_body = {
    "mappings": {
        "properties": {
            "category_name": {"type": "keyword"},
            "category_embedding": {
                "type": "dense_vector",
                "dims": 768  # Embedding size for profile categories
            }
        }
    }
}

if not es.indices.exists(index=profile_knowledge_base_index):
    es.indices.create(index=profile_knowledge_base_index, body=profile_knowledge_base_body)
    print(f"Index '{profile_knowledge_base_index}' created successfully!")
else:
    print(f"Index '{profile_knowledge_base_index}' already exists.")

# Create news genre index with vector field for embeddings
news_genre_index_body = {
    "mappings": {
        "properties": {
            "genre_name": {"type": "keyword"},
            "genre_embedding": {
                "type": "dense_vector",
                "dims": 768  # Embedding size for news genre categories
            }
        }
    }
}

if not es.indices.exists(index=news_genre_index):
    es.indices.create(index=news_genre_index, body=news_genre_index_body)
    print(f"Index '{news_genre_index}' created successfully!")
else:
    print(f"Index '{news_genre_index}' already exists.")

# Load a pre-trained model and tokenizer for embedding generation
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set pad_token as eos_token if pad_token doesn't exist

model = AutoModel.from_pretrained(model_name)

# Load a pre-trained LLM (e.g., GPT-2 or any causal LM for text generation)
llm_model_name = "gpt2"  # Example: Using GPT-2 for text generation
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name)
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

# Function to get embedding of a text (transaction or category)
def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # The attention mask is already part of the 'inputs' dict, so no need to pass it again.
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Use mean of last hidden layer
    return embeddings

# Function to recommend a news genre based on user profile using cosine similarity
def recommend_news_based_on_profile(user_transactions_embedding, profile_embeddings, genre_data):
    # Check that the profile_embeddings list is not empty
    if not profile_embeddings:
        print("No profile embeddings found!")
        return None

    # Find the most similar profile using cosine similarity
    similarities = cosine_similarity([user_transactions_embedding], profile_embeddings)
    most_similar_profile_idx = np.argmax(similarities)

    # Get the category name
    recommended_profile = profile_categories[most_similar_profile_idx]

    # Print the user profile categorization
    print(f"The user's profile category is: {recommended_profile}")

    # Fetch all news genre embeddings from Elasticsearch for comparison
    res = es.search(index=news_genre_index, body={"query": {"match_all": {}}})

    if not res['hits']['hits']:
        print("No news genres found!")
        return None

    # Get genre embeddings and names
    genre_embeddings = []
    genre_names = []
    for hit in res['hits']['hits']:
        genre_name = hit['_source']['genre_name']
        genre_embedding = np.array(hit['_source']['genre_embedding'])
        genre_embeddings.append(genre_embedding)
        genre_names.append(genre_name)

    # Calculate cosine similarities between the profile embedding and all genre embeddings
    genre_similarities = cosine_similarity([profile_embeddings[most_similar_profile_idx]], genre_embeddings)

    # Find the most similar genre
    most_similar_genre_idx = np.argmax(genre_similarities)
    recommended_genre = genre_names[most_similar_genre_idx]

    print(f"Recommended News Genre: {recommended_genre}")

    # Use LLM to generate a relevant recommendation for the user
    prompt = f"Based on the user's profile as a '{recommended_profile}', recommend news related to '{recommended_genre}'."

    inputs = llm_tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = llm_model.generate(inputs['input_ids'], max_length=50, num_return_sequences=1)

    recommendation = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return recommendation

# Sample dataset of 50 transactions (as mentioned previously)
sample_transactions = [
    "Bought a new smartphone from the electronics store",
    "Purchased groceries from Walmart",
    "Subscription to Netflix for the month",
    "Bought a new gaming console from Best Buy",
    "Purchased a laptop for work",
    "Paid for Spotify premium subscription",
    "Ordered a pizza from Domino's",
    "Bought a pair of running shoes from Nike",
    "Paid for a gym membership",
    "Subscription to Amazon Prime",
    "Bought a set of headphones for music",
    "Purchased a kitchen blender from Amazon",
    "Paid for a monthly digital magazine subscription",
    "Bought a set of books on Amazon",
    "Ordered a coffee from Starbucks",
    "Subscription to an online learning platform",
    "Bought a smartwatch from an electronics store",
    "Ordered a new backpack for travel",
    "Paid for an Uber ride to the airport",
    "Bought a gift card for a friend",
    "Paid for a meal at a fancy restaurant",
    "Bought movie tickets for a new release",
    "Bought groceries from Whole Foods",
    "Purchased a new pair of sunglasses",
    "Paid for an online course in data science",
    "Bought a new television",
    "Bought a subscription to Disney+",
    "Paid for a hotel booking during vacation",
    "Purchased a new camera for photography",
    "Bought a set of cooking tools from Amazon",
    "Paid for a personal development course",
    "Bought an expensive bottle of wine from a winery",
    "Subscription to a meditation app",
    "Paid for a monthly meal kit delivery",
    "Ordered a new set of furniture for the living room",
    "Bought a Bluetooth speaker for the house",
    "Paid for a car rental during travel",
    "Ordered a smartwatch band for replacement",
    "Paid for an annual conference ticket",
    "Bought a set of professional work clothes",
    "Purchased a new tablet for reading",
    "Bought a bike for commuting",
    "Paid for a mobile data plan",
    "Subscription to a health and fitness app",
    "Bought a new printer for the home office",
    "Paid for a flight booking for vacation",
    "Bought a set of art supplies for painting",
    "Purchased a new set of golf clubs",
    "Bought a family membership at the local zoo",
    "Paid for a music concert ticket",
    "Bought a membership for a photography club",
    "Paid for an online cooking class",
    "Bought a set of kitchen gadgets for convenience"
]

# Embed and index the transactions
for i, transaction in enumerate(sample_transactions):
    transaction_embedding = get_text_embedding(transaction)
    doc = {
        "user_id": f"user_{i+1}",
        "transaction_text": transaction,
        "transaction_embedding": transaction_embedding.tolist()  # Convert numpy array to list
    }
    es.index(index=transactions_index, body=doc)

# Sample profile data for categorizing users
profile_categories = ['Technology Enthusiast', 'Food Lover', 'Fitness Buff', 'Entertainment Aficionado', 'Shopper']
profile_embeddings = []
for category in profile_categories:
    profile_embedding = get_text_embedding(category)
    profile_embeddings.append(profile_embedding)

# Use the embeddings for recommendation
user_transaction_embeddings = get_text_embedding("Bought a new smartphone from the electronics store")

# Recommend news genre
recommendation = recommend_news_based_on_profile(user_transaction_embeddings, profile_embeddings, sample_transactions)
print(f"Final Recommendation: {recommendation}")


Index 'transactions_index' already exists.
Index 'profiles_index' already exists.
Index 'profile_knowledge_base_index' already exists.
Index 'news_genre_index' already exists.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The user's profile category is: Shopper
Recommended News Genre: Food
Final Recommendation: Based on the user's profile as a 'Shopper', recommend news related to 'Food'.

The user's profile is not a 'Shopper' and is not a 'Food'

The user's profile is not a


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# Sample user profile and news categories (for simplicity, we're using random vectors)
np.random.seed(42)

# Create a mock user profile (vector)
user_profile_vector = np.random.rand(1, 5)  # Represents 5 profile categories

# Example of past transactions (vector representation)
user_transactions_vector = np.random.rand(1, 5)  # Represents 5 transaction categories

# News Index (5 categories of news)
news_index = np.random.rand(5, 5)  # Representing 5 news categories

# Combine profile and transaction vectors for the final user profile
user_combined_vector = (user_profile_vector + user_transactions_vector) / 2  # Averaging

# Cosine similarity function to find similarity between user profile and news categories
def get_news_recommendations(user_vector, news_index):
    # Compute the cosine similarity between the user vector and the news index
    similarity_scores = cosine_similarity(user_vector, news_index)
    return similarity_scores

# Get similarity scores for news categories
similarity_scores = get_news_recommendations(user_combined_vector, news_index)

# Sort and recommend the most similar news categories based on the scores
top_news_categories = similarity_scores.argsort()[0][-3:]  # Top 3 news categories

# Mapping of news categories for interpretation
news_categories = ["Sports", "Politics", "Tech", "Entertainment", "Health"]

# Generate a recommendation based on the top categories
recommended_news = [news_categories[i] for i in top_news_categories]

# Display the recommended news genres
print("Recommended News Genres based on your profile and past transactions:")
for genre in recommended_news:
    print(genre)

# Optional: Integrate LLM (e.g., GPT-4) for better understanding and refinement
def use_llm_for_refinement(user_profile, recommended_genres):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")

    # Construct a prompt based on user's profile and recommended genres
    prompt = f"User profile indicates interest in {user_profile}. Based on this, the most suitable news genres are {recommended_genres}. Please refine the recommendations."

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

    refined_recommendations = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return refined_recommendations

# Refine recommendations using LLM (Optional)
refined_recommendations = use_llm_for_refinement(user_combined_vector, recommended_news)
print("Refined News Genre Recommendations based on LLM:")
print(refined_recommendations)

Recommended News Genres based on your profile and past transactions:
Tech
Sports
Politics


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Refined News Genre Recommendations based on LLM:
User profile indicates interest in [[0.26526732 0.50439896 0.79908504 0.59988675 0.43204561]]. Based on this, the most suitable news genres are ['Tech', 'Sports', 'Politics']. Please refine the recommendations.

[0.26526732 0.50439896 0.79908504 0.59988675 0.43204561]]. Based on this, the most suitable news genres are


In [ ]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Initialize the SentenceTransformer model for generating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can change the model if needed

# Create Indexes for Transactions, User Profiles, and News Genres

def create_transactions_index():
    transaction_mapping = {
        "mappings": {
            "properties": {
                "transaction_id": {"type": "integer"},
                "user_id": {"type": "integer"},
                "product_category": {"type": "text"},
                "transaction_amount": {"type": "float"},
                "transaction_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Dimensionality of the embedding vectors (same as SentenceTransformer)
                }
            }
        }
    }

    if not es.indices.exists(index="user_transactions"):
        es.indices.create(index="user_transactions", body=transaction_mapping)
        print("Transactions index created.")

def create_user_profile_index():
    user_profile_mapping = {
        "mappings": {
            "properties": {
                "user_id": {"type": "integer"},
                "profile_embedding": {
                    "type": "dense_vector",
                    "dims": 384  # Dimensionality of the embedding vectors (same as SentenceTransformer)
                }
            }
        }
    }

    if not es.indices.exists(index="user_profiles"):
        es.indices.create(index="user_profiles", body=user_profile_mapping)
        print("User Profiles index created.")

def create_news_genre_index():
    news_genre_mapping = {
        "mappings": {
            "properties": {
                "genre": {"type": "keyword"},
                "embedding": {
                    "type": "dense_vector",
                    "dims": 384  # Dimensionality of the embedding vectors
                }
            }
        }
    }

    if not es.indices.exists(index="news_genres"):
        es.indices.create(index="news_genres", body=news_genre_mapping)
        print("News Genres index created.")

# Function to generate embeddings for categories
def generate_category_embedding(category):
    return model.encode(f"Category: {category}")

# Index Sample Transactions with Vectorized Product Categories
def index_sample_transactions():
    transactions = [
        {"transaction_id": 1, "user_id": 1, "product_category": "Tech", "transaction_amount": 150.5},
        {"transaction_id": 2, "user_id": 1, "product_category": "Tech", "transaction_amount": 300.2},
        {"transaction_id": 3, "user_id": 1, "product_category": "Sports", "transaction_amount": 50.0},
        {"transaction_id": 4, "user_id": 1, "product_category": "Entertainment", "transaction_amount": 75.0},
        {"transaction_id": 5, "user_id": 1, "product_category": "Politics", "transaction_amount": 120.0},
        {"transaction_id": 6, "user_id": 1, "product_category": "Tech", "transaction_amount": 200.3},
        {"transaction_id": 7, "user_id": 1, "product_category": "Sports", "transaction_amount": 60.0},
        {"transaction_id": 8, "user_id": 1, "product_category": "Entertainment", "transaction_amount": 90.0},
        {"transaction_id": 9, "user_id": 1, "product_category": "Health", "transaction_amount": 30.0},
        {"transaction_id": 10, "user_id": 1, "product_category": "Tech", "transaction_amount": 180.0}
    ]

    for transaction in transactions:
        category_embedding = generate_category_embedding(transaction["product_category"])
        es.index(index="user_transactions", document={
            "transaction_id": transaction["transaction_id"],
            "user_id": transaction["user_id"],
            "product_category": transaction["product_category"],
            "transaction_amount": transaction["transaction_amount"],
            "transaction_vector": category_embedding.tolist()
        })
    print("Transaction data indexed with vectors into Elasticsearch.")

# Function to generate the user profile embedding
def generate_user_profile_embedding(user_id):
    query = {
        "query": {
            "term": {
                "user_id": user_id
            }
        }
    }
    response = es.search(index="user_transactions", body=query)

    transaction_vectors = [hit["_source"]["transaction_vector"] for hit in response["hits"]["hits"]]

    if transaction_vectors:
        user_profile_embedding = np.mean(transaction_vectors, axis=0)
    else:
        user_profile_embedding = np.zeros(384)  # Default to zero vector if no transactions

    return user_profile_embedding

# Index User Profile Embedding
def index_user_profile(user_id, profile_embedding):
    es.index(index="user_profiles", id=user_id, document={
        "user_id": user_id,
        "profile_embedding": profile_embedding.tolist()  # Convert to list for storage
    })
    print(f"User profile for user_id={user_id} indexed with vector embedding.")

# Function to index news genres with their embeddings
def index_news_genres():
    news_genres = ["Tech", "Sports", "Politics", "Entertainment", "Health"]

    for genre in news_genres:
        genre_embedding = generate_category_embedding(genre)
        es.index(index="news_genres", document={
            "genre": genre,
            "embedding": genre_embedding.tolist()
        })
    print("News genres indexed with vector embeddings.")

# Function to recommend the most relevant news genre for a user based on profile
def recommend_news_genre(user_profile_embedding):
    query = {
        "size": 1000,  # You can adjust this number based on your dataset size
        "_source": ["genre", "embedding"]
    }
    response = es.search(index="news_genres", body=query)

    genre_embeddings = []
    for hit in response["hits"]["hits"]:
        genre = hit["_source"]["genre"]
        embedding = np.array(hit["_source"]["embedding"])
        genre_embeddings.append((genre, embedding))

    similarities = []
    for genre, embedding in genre_embeddings:
        sim = cosine_similarity([user_profile_embedding], [embedding])[0][0]
        similarities.append((genre, sim))

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_genre = similarities[0][0]

    return most_relevant_genre

# Main workflow
def main():
    # Create all indices
    create_transactions_index()
    create_user_profile_index()
    create_news_genre_index()

    # Index sample transactions
    index_sample_transactions()

    # Generate and index user profile embedding
    user_profile_embedding = generate_user_profile_embedding(user_id=1)
    index_user_profile(user_id=1, profile_embedding=user_profile_embedding)

    # Index predefined news genres with embeddings
    index_news_genres()

    # Recommend a news genre for the user based on their profile
    recommended_genre = recommend_news_genre(user_profile_embedding)
    print(f"Recommended news genre for user_id=1: {recommended_genre}")

# Run the main workflow
if __name__ == "__main__":
    main()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Transactions index created.
User Profiles index created.
News Genres index created.
Transaction data indexed with vectors into Elasticsearch.
User profile for user_id=1 indexed with vector embedding.
News genres indexed with vector embeddings.
Recommended news genre for user_id=1: Tech


In [ ]:
# Initialize the SentenceTransformer model for generating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can change the model if needed

# Create Indexes for Transactions, User Profiles, News Genres

def create_transactions_index():
    transaction_mapping = {
        "mappings": {
            "properties": {
                "transaction_id": {"type": "integer"},
                "user_id": {"type": "integer"},
                "product_name": {"type": "text"},
                "transaction_amount": {"type": "float"},
                "transaction_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Dimensionality of the embedding vectors (same as SentenceTransformer)
                }
            }
        }
    }

    if not es.indices.exists(index="user_transactions"):
        es.indices.create(index="user_transactions", body=transaction_mapping)
        print("Transactions index created.")

def create_user_profile_index():
    user_profile_mapping = {
        "mappings": {
            "properties": {
                "user_id": {"type": "integer"},
                "profile_embedding": {
                    "type": "dense_vector",
                    "dims": 384  # Dimensionality of the embedding vectors (same as SentenceTransformer)
                }
            }
        }
    }

    if not es.indices.exists(index="user_profiles"):
        es.indices.create(index="user_profiles", body=user_profile_mapping)
        print("User Profiles index created.")

def create_news_genre_index():
    news_genre_mapping = {
        "mappings": {
            "properties": {
                "genre": {"type": "keyword"},
                "embedding": {
                    "type": "dense_vector",
                    "dims": 384  # Dimensionality of the embedding vectors
                }
            }
        }
    }

    if not es.indices.exists(index="news_genres"):
        es.indices.create(index="news_genres", body=news_genre_mapping)
        print("News Genres index created.")

# Function to generate embeddings for product names
def generate_product_name_embedding(product_name):
    return model.encode(f"Product: {product_name}")

# Index Sample Transactions with Vectorized Product Names
def index_sample_transactions():
    transactions = [
        {"transaction_id": 1, "user_id": 1, "product_name": "iPhone 14", "transaction_amount": 1505.5},
        {"transaction_id": 2, "user_id": 1, "product_name": "MacBook Pro 16", "transaction_amount": 3002.2},
        {"transaction_id": 3, "user_id": 1, "product_name": "Nike Air Max", "transaction_amount": 150.0},
        {"transaction_id": 4, "user_id": 1, "product_name": "Sony Headphones", "transaction_amount": 175.0},
        {"transaction_id": 5, "user_id": 1, "product_name": "LG TV 55 inch", "transaction_amount": 1200.0},
        {"transaction_id": 6, "user_id": 1, "product_name": "iPad Pro", "transaction_amount": 1050.3},
        {"transaction_id": 7, "user_id": 1, "product_name": "Samsung Galaxy S22", "transaction_amount": 850.0},
        {"transaction_id": 8, "user_id": 1, "product_name": "Xbox Series X", "transaction_amount": 499.0},
        {"transaction_id": 9, "user_id": 1, "product_name": "Bose SoundLink", "transaction_amount": 199.0},
        {"transaction_id": 10, "user_id": 1, "product_name": "Apple Watch Series 7", "transaction_amount": 399.0}
    ]

    for transaction in transactions:
        product_embedding = generate_product_name_embedding(transaction["product_name"])
        es.index(index="user_transactions", document={
            "transaction_id": transaction["transaction_id"],
            "user_id": transaction["user_id"],
            "product_name": transaction["product_name"],
            "transaction_amount": transaction["transaction_amount"],
            "transaction_vector": product_embedding.tolist()
        })
    print("Transaction data indexed with vectors into Elasticsearch.")

# Function to generate the user profile embedding
def generate_user_profile_embedding(user_id):
    query = {
        "query": {
            "term": {
                "user_id": user_id
            }
        }
    }
    response = es.search(index="user_transactions", body=query)

    transaction_vectors = [hit["_source"]["transaction_vector"] for hit in response["hits"]["hits"]]

    if transaction_vectors:
        user_profile_embedding = np.mean(transaction_vectors, axis=0)
    else:
        user_profile_embedding = np.zeros(384)  # Default to zero vector if no transactions

    return user_profile_embedding

# Index User Profile Embedding
def index_user_profile(user_id, profile_embedding):
    es.index(index="user_profiles", id=user_id, document={
        "user_id": user_id,
        "profile_embedding": profile_embedding.tolist()  # Convert to list for storage
    })
    print(f"User profile for user_id={user_id} indexed with vector embedding.")

# Function to generate category embeddings for the 5 predefined categories
def generate_category_embedding(category):
    return model.encode(f"Category: {category}")

# Function to index predefined user categories
def index_user_categories():
    categories = ["Fitness", "Political", "Bimbo", "Smart aleck", "Influencer"]

    for category in categories:
        category_embedding = generate_category_embedding(category)
        es.index(index="user_categories", document={
            "category": category,
            "embedding": category_embedding.tolist()
        })
    print("User profile categories indexed with embeddings.")

# Function to recommend the most relevant category for a user based on profile
def recommend_user_category(user_profile_embedding):
    query = {
        "size": 1000,  # You can adjust this number based on your dataset size
        "_source": ["category", "embedding"]
    }
    response = es.search(index="user_categories", body=query)

    # If no categories are found, return a default category
    if not response["hits"]["hits"]:
        return "Default Category"  # Fallback to a default category

    category_embeddings = []
    for hit in response["hits"]["hits"]:
        category = hit["_source"]["category"]
        embedding = np.array(hit["_source"]["embedding"])
        category_embeddings.append((category, embedding))

    similarities = []
    for category, embedding in category_embeddings:
        sim = cosine_similarity([user_profile_embedding], [embedding])[0][0]
        similarities.append((category, sim))

    if not similarities:
        return "Default Category"  # If no similarities found, return a default category

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_category = similarities[0][0]

    return most_relevant_category

# Function to index news genres with their embeddings
def index_news_genres():
    news_genres = ["Tech", "Sports", "Politics", "Drama", "Health"]

    for genre in news_genres:
        genre_embedding = generate_category_embedding(genre)
        es.index(index="news_genres", document={
            "genre": genre,
            "embedding": genre_embedding.tolist()
        })
    print("News genres indexed with vector embeddings.")

# Function to recommend the most relevant news genre for a user based on profile
def recommend_news_genre(user_profile_embedding):
    query = {
        "size": 1000,  # You can adjust this number based on your dataset size
        "_source": ["genre", "embedding"]
    }
    response = es.search(index="news_genres", body=query)

    # If no genres are found, return a default genre
    if not response["hits"]["hits"]:
        return "Default Genre"  # Fallback to a default genre

    genre_embeddings = []
    for hit in response["hits"]["hits"]:
        genre = hit["_source"]["genre"]
        embedding = np.array(hit["_source"]["embedding"])
        genre_embeddings.append((genre, embedding))

    similarities = []
    for genre, embedding in genre_embeddings:
        sim = cosine_similarity([user_profile_embedding], [embedding])[0][0]
        similarities.append((genre, sim))

    if not similarities:
        return "Default Genre"  # If no similarities found, return a default genre

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_genre = similarities[0][0]

    return most_relevant_genre

# Main workflow
def main():
    # Create all indices
    create_transactions_index()
    create_user_profile_index()
    create_news_genre_index()

    # Index sample transactions
    index_sample_transactions()

    # Generate and index user profile embedding
    user_profile_embedding = generate_user_profile_embedding(user_id=1)
    index_user_profile(user_id=1, profile_embedding=user_profile_embedding)

    # Index predefined user categories with embeddings
    index_user_categories()

    # Recommend a user profile category based on their transaction data
    recommended_category = recommend_user_category(user_profile_embedding)
    print(f"Recommended user profile category for user_id=1: {recommended_category}")

    # Index predefined news genres with embeddings
    index_news_genres()

    # Recommend a news genre for the user based on their profile
    recommended_genre = recommend_news_genre(user_profile_embedding)
    print(f"Recommended news genre for user_id=1: {recommended_genre}")

# Run the main workflow
if __name__ == "__main__":
    main()

Transactions index created.
User Profiles index created.
News Genres index created.
Transaction data indexed with vectors into Elasticsearch.
User profile for user_id=1 indexed with vector embedding.
User profile categories indexed with embeddings.
Recommended user profile category for user_id=1: Default Category
News genres indexed with vector embeddings.
Recommended news genre for user_id=1: Default Genre


In [ ]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



# Initialize SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose another model if needed

# Sample Transaction Data
transactions = [
    {"description": "running shoes", "cost": 150.0},
    {"description": "gym membership", "cost": 150.0},
    {"description": "badminton shoes", "cost": 150.0},
    {"description": "Jay Chou concert", "cost": 150.0}

]

# Predefined Profile Categories (more abstract categories, not same as genres)
categories = [
    {"category": "Fitness guru", "description": "Includes activities related to sports, exercise, fitness, well-being, and personal care. Also include buying sports,exercise related items, gym memberships, health tracking products"},
    {"category": "Star Chaser", "description": "Someone thats like to go to events like concerts, movies, and art-related activities, also likes to buy boy/girl group merchandises or products that have their favourite idol as the brand ambassador"},
    {"category": "Politics and Society", "description": "Includes political discussions, debates, government policies, and social issues."},
    {"category": "Change the world guy", "description": "Includes learning resources, online courses, educational content, and tech developments, and anything innovative or new discoveries that would change the world."},
    {"category": "Financial freedom lo", "description": "Includes anything related to trading, stocks, options, investments, prices of gold,silver, investment products, housing, crypto."}
]

# Predefined News Genres with Descriptions
news_genres = [
    {"genre": "Entertainment", "description": "Covers movies, music, celebrity news, TV shows, and all things related to entertainment."},
    {"genre": "Politics", "description": "In-depth coverage of political news, elections, government policies, and political events."},
    {"genre": "Sports", "description": "Focuses on sports events, matches, player interviews, and all things related to sports."},
    {"genre": "Education", "description": "Includes educational content, online courses, academic developments, and learning resources."},
    {"genre": "Technology", "description": "Covers tech news, gadgets, innovations, software, and the world of technology."}
]

# Create Elasticsearch Indices
def create_transaction_index():
    transaction_mapping = {
        "mappings": {
            "properties": {
                "description": {"type": "text"},
                "cost": {"type": "float"},
                "transaction_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_transactions"):
        es.indices.create(index="user_transactions", body=transaction_mapping)
        print("Transactions index created.")

def create_category_index():
    category_mapping = {
        "mappings": {
            "properties": {
                "category": {"type": "keyword"},
                "category_description": {"type": "text"},
                "category_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_categories"):
        es.indices.create(index="user_categories", body=category_mapping)
        print("Category index created.")

def create_news_genre_index():
    news_genre_mapping = {
        "mappings": {
            "properties": {
                "genre": {"type": "keyword"},
                "genre_description": {"type": "text"},
                "genre_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_news_genres"):
        es.indices.create(index="user_news_genres", body=news_genre_mapping)
        print("News genres index created.")

# Generate Embeddings for Transaction Descriptions
def generate_transaction_embedding(description):
    return model.encode(description)

# Generate Embeddings for Category Descriptions
def generate_category_description_embedding(description):
    return model.encode(description)

# Generate Embeddings for News Genres Descriptions
def generate_news_genre_embedding(description):
    return model.encode(description)

# Index Sample Transactions
def index_transactions():
    for transaction in transactions:
        description = transaction["description"]
        embedding = generate_transaction_embedding(description)

        es.index(index="user_transactions", document={
            "description": description,
            "cost": transaction["cost"],
            "transaction_vector": embedding.tolist()
        })
    print("Transactions indexed.")

# Index Profile Categories with Descriptions
def index_categories():
    for category in categories:
        embedding = generate_category_description_embedding(category["description"])

        es.index(index="user_categories", document={
            "category": category["category"],
            "category_description": category["description"],
            "category_vector": embedding.tolist()
        })
    print("Profile categories indexed.")

# Index News Genres with Descriptions
def index_news_genres():
    for genre in news_genres:
        embedding = generate_news_genre_embedding(genre["description"])

        es.index(index="user_news_genres", document={
            "genre": genre["genre"],
            "genre_description": genre["description"],
            "genre_vector": embedding.tolist()
        })
    print("News genres indexed.")

# Recommend Profile Category for a User
def recommend_profile_category(user_transactions):
    # Aggregate all user's transactions into a single embedding (using the entire list of transactions)
    user_embeddings = [generate_transaction_embedding(t["description"]) for t in user_transactions]
    user_profile_vector = np.mean(user_embeddings, axis=0)

    # Fetch all category embeddings from the index
    query = {
        "size": 1000,  # You can adjust this number based on your dataset size
        "_source": ["category", "category_description", "category_vector"]
    }
    response = es.search(index="user_categories", body=query)

    # Ensure we have results
    if not response["hits"]["hits"]:
        return "Default Category"

    category_embeddings = []
    for hit in response["hits"]["hits"]:
        category = hit["_source"]["category"]
        embedding = np.array(hit["_source"]["category_vector"])
        category_embeddings.append((category, embedding))

    similarities = []
    for category, embedding in category_embeddings:
        sim = cosine_similarity([user_profile_vector], [embedding])[0][0]
        similarities.append((category, sim))

    if not similarities:
        return "Default Category"  # If no similarities found, return a default category

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_category = similarities[0][0]

    return most_relevant_category

# Recommend News Genre based on Profile Category
def recommend_news_genre_based_on_profile(category):
    # Fetch all genre embeddings from the news genre index
    query = {
        "size": 1000,  # You can adjust this number based on your dataset size
        "_source": ["genre", "genre_description", "genre_vector"]
    }
    response = es.search(index="user_news_genres", body=query)

    # Ensure we have results
    if not response["hits"]["hits"]:
        return "Default Genre"

    genre_embeddings = []
    for hit in response["hits"]["hits"]:
        genre = hit["_source"]["genre"]
        genre_description = hit["_source"]["genre_description"]
        embedding = np.array(hit["_source"]["genre_vector"])
        genre_embeddings.append((genre, genre_description, embedding))

    # Match category description to the genre descriptions
    category_embedding = generate_category_description_embedding(category)

    similarities = []
    for genre, genre_description, embedding in genre_embeddings:
        genre_similarity = cosine_similarity([category_embedding], [embedding])[0][0]
        similarities.append((genre, genre_similarity))

    if not similarities:
        return "Default Genre"  # If no similarities found, return a default genre

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_genre = similarities[0][0]

    return most_relevant_genre

# Main function to run the workflow
def main():
    # Step 1: Create indices
    create_transaction_index()
    create_category_index()
    create_news_genre_index()

    # Step 2: Index sample transactions, categories, and news genres
    index_transactions()
    index_categories()
    index_news_genres()

    # Step 3: Recommend a profile category based on **all** user transactions
    # Using all the transactions from the `transactions` list (instead of a sample)
    user_transactions = transactions  # Now using all transactions in the list

    # Step 4: Recommend a news genre based on user profile
    recommended_category = recommend_profile_category(user_transactions)
    print(f"Recommended user profile category: {recommended_category}")

    recommended_genre = recommend_news_genre_based_on_profile(recommended_category)
    print(f"Recommended news genre: {recommended_genre}")

# Run the main function
if __name__ == "__main__":
    main()

Transactions indexed.
Profile categories indexed.
News genres indexed.
Recommended user profile category: Fitness guru
Recommended news genre: Sports


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

# Initialize Elasticsearch and the Sentence Transformer model
es = Elasticsearch()
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample Transactions (assuming it's in the form of a list of dictionaries)
transactions = [
    {"user_id": 1, "description": "running shoes", "cost": 150.0},
    {"user_id": 1, "description": "gym membership", "cost": 150.0},
    {"user_id": 1, "description": "protein powder", "cost": 150.0},
    {"user_id": 1, "description": "bcaa", "cost": 150.0},
    {"user_id": 1, "description": "lifting straps", "cost": 150.0},
    {"user_id": 1, "description": "shaker bottle", "cost": 150.0},
    {"user_id": 2, "description": "badminton shoes", "cost": 150.0},
    {"user_id": 2, "description": "Jay Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "Jay Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "IVE music album", "cost": 150.0},
    {"user_id": 3, "description": "IU photocards", "cost": 150.0},
    {"user_id": 3, "description": "New Jeans merchandise", "cost": 150.0},
    {"user_id": 3, "description": "Twice concert ticket", "cost": 150.0},
    {"user_id": 3, "description": "Limited edition shoes endorsed by G-dragon", "cost": 150.0},
    {"user_id": 3, "description": "Eric Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "Waterbomb music festival", "cost": 150.0},
    {"user_id": 3, "description": "Eric Chou fan sign event", "cost": 150.0},
    {"user_id": 3, "description": "Aespa photobook", "cost": 150.0}
]

# Predefined Profile Categories (same as before)
categories = [
    {"category": "Fitness guru", "description": "A Fitness Guru is someone who prioritizes physical fitness and well-being, often engaging in regular workouts, sports activities, and wellness practices. They are passionate about maintaining an active lifestyle and improving their health through exercise, nutrition, and self-care."},
    {"category": "Star Chaser", "description": "A Pop Star Chaser is someone who is deeply fascinated by celebrities, particularly pop stars, and tailors their lifestyle and spending habits around their favorite idols. They follow the latest news, social media updates, and public appearances of pop stars, attending concerts, meet-and-greet events, and exclusive fan gatherings."},
    {"category": "Politics and Society", "description": "Includes political discussions, debates, government policies, and social issues."},
    {"category": "Change the world guy", "description": "Includes learning resources, online courses, educational content, and tech developments, and anything innovative or new discoveries that would change the world."},
    {"category": "Financial freedom lo", "description": "Includes anything related to trading, stocks, options, investments, prices of gold, silver, investment products, housing, crypto."}
]

# Predefined News Genres (same as before)
news_genres = [
    {"genre": "Entertainment", "description": "Covers movies, music, celebrity news, TV shows, and all things related to entertainment."},
    {"genre": "Politics", "description": "In-depth coverage of political news, elections, government policies, and political events."},
    {"genre": "Sports", "description": "Focuses on covering various sports events, matches, player interviews, and athletic achievements."},
    {"genre": "Health & Fitness", "description": "Focuses on the latest developments in exercise, nutrition, wellness trends, and overall well-being."},
    {"genre": "Technology", "description": "Covers tech news, gadgets, innovations, software, and the world of technology."}
]

# Elasticsearch Indexing Functions

def create_transaction_index():
    transaction_mapping = {
        "mappings": {
            "properties": {
                "user_id": {"type": "keyword"},
                "description": {"type": "text"},
                "cost": {"type": "float"},
                "transaction_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_transactions"):
        es.indices.create(index="user_transactions", body=transaction_mapping)
        print("Transactions index created.")

def create_category_index():
    category_mapping = {
        "mappings": {
            "properties": {
                "category": {"type": "keyword"},
                "category_description": {"type": "text"},
                "category_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_categories"):
        es.indices.create(index="user_categories", body=category_mapping)
        print("Category index created.")

def create_news_genre_index():
    news_genre_mapping = {
        "mappings": {
            "properties": {
                "genre": {"type": "keyword"},
                "genre_description": {"type": "text"},
                "genre_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_news_genres"):
        es.indices.create(index="user_news_genres", body=news_genre_mapping)
        print("News genres index created.")

# Generate Embeddings for Transaction Descriptions
def generate_transaction_embedding(description):
    return model.encode(description)

# Generate Embeddings for Category Descriptions
def generate_category_description_embedding(description):
    return model.encode(description)

# Generate Embeddings for News Genres Descriptions
def generate_news_genre_embedding(description):
    return model.encode(description)

# Index Sample Transactions
def index_transactions():
    for transaction in transactions:
        description = transaction["description"]
        user_id = transaction["user_id"]  # Extract user_id
        embedding = generate_transaction_embedding(description)

        es.index(index="user_transactions", document={
            "user_id": user_id,  # Include user_id in the document
            "description": description,
            "cost": transaction["cost"],
            "transaction_vector": embedding.tolist()
        })
    print("Transactions indexed.")

# Index Profile Categories with Descriptions
def index_categories():
    for category in categories:
        embedding = generate_category_description_embedding(category["description"])

        es.index(index="user_categories", document={
            "category": category["category"],
            "category_description": category["description"],
            "category_vector": embedding.tolist()
        })
    print("Profile categories indexed.")

# Index News Genres with Descriptions
def index_news_genres():
    for genre in news_genres:
        embedding = generate_news_genre_embedding(genre["description"])

        es.index(index="user_news_genres", document={
            "genre": genre["genre"],
            "genre_description": genre["description"],
            "genre_vector": embedding.tolist()
        })
    print("News genres indexed.")

# Step 1: Aggregate a User's Transactions
def aggregate_user_transactions(user_id):
    # Filter transactions by user_id
    user_transactions = [t["description"] for t in transactions if t["user_id"] == user_id]

    # Combine all transaction descriptions for the user into one string
    aggregated_description = " ".join(user_transactions)

    return aggregated_description

# Step 2: Find the most relevant category based on aggregated transaction descriptions
def find_relevant_category(user_id):
    aggregated_description = aggregate_user_transactions(user_id)

    # Generate embedding for aggregated description
    user_embedding = generate_transaction_embedding(aggregated_description)

    # Query Elasticsearch for categories and calculate cosine similarity with category embeddings
    query = {
        "size": 1000,  # Fetch all categories
        "_source": ["category", "category_description", "category_vector"]
    }
    response = es.search(index="user_categories", body=query)

    category_embeddings = []
    for hit in response["hits"]["hits"]:
        category = hit["_source"]["category"]
        category_description = hit["_source"]["category_description"]
        embedding = np.array(hit["_source"]["category_vector"])
        category_embeddings.append((category, category_description, embedding))

    # Find the most relevant category based on cosine similarity
    similarities = []
    for category, category_description, embedding in category_embeddings:
        similarity = cosine_similarity([user_embedding], [embedding])[0][0]
        similarities.append((category, similarity))

    # Return the category with the highest similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_category = similarities[0][0]

    return most_relevant_category

# Step 3: Find the most relevant news genre based on the selected category's embedding
def find_relevant_news_genre(category):
    category_description = next(c['description'] for c in categories if c['category'] == category)
    category_embedding = generate_category_description_embedding(category_description)

    query = {
        "size": 1000,  # Fetch all genres
        "_source": ["genre", "genre_description", "genre_vector"]
    }
    response = es.search(index="user_news_genres", body=query)

    genre_embeddings = []
    for hit in response["hits"]["hits"]:
        genre = hit["_source"]["genre"]
        genre_description = hit["_source"]["genre_description"]
        embedding = np.array(hit["_source"]["genre_vector"])
        genre_embeddings.append((genre, genre_description, embedding))

    similarities = []
    for genre, genre_description, embedding in genre_embeddings:
        similarity = cosine_similarity([category_embedding], [embedding])[0][0]
        similarities.append((genre, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_genre = similarities[0][0]

    return most_relevant_genre

# Main function to run the workflow
def main(user_id):
    create_transaction_index()
    create_category_index()
    create_news_genre_index()

    # Index sample transactions, categories, and news genres
    index_transactions()
    index_categories()
    index_news_genres()

    # Find the relevant category and news genre for the specified user
    relevant_category = find_relevant_category(user_id)
    print(f"Relevant Category for User {user_id}: {relevant_category}")

    relevant_genre = find_relevant_news_genre(relevant_category)
    print(f"Relevant News Genre for User {user_id}: {relevant_genre}")

# Run the main function with user_id = 1 as an example
if __name__ == "__main__":
    main(user_id=1)


Transactions indexed.
Profile categories indexed.
News genres indexed.
Relevant Category for Transaction: Fitness guru
Relevant News Genre: Health & Fitness


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

# Initialize Elasticsearch and the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample Transactions (assuming it's in the form of a list of dictionaries)
transactions = [
    {"user_id": 2, "description": "reusable water bottle", "cost": 20.0},
    {"user_id": 2, "description": "organic cotton t-shirt", "cost": 25.0},
    {"user_id": 2, "description": "solar-powered charger", "cost": 50.0},
    {"user_id": 2, "description": "bamboo toothbrush", "cost": 5.0},
    {"user_id": 2, "description": "composting bin", "cost": 40.0},
    {"user_id": 2, "description": "eco-friendly laundry detergent", "cost": 15.0},
    {"user_id": 2, "description": "recycled paper notebooks", "cost": 10.0},
    {"user_id": 2, "description": "plant-based protein powder", "cost": 30.0},
    {"user_id": 2, "description": "electric car charging station subscription", "cost": 100.0},
    {"user_id": 2, "description": "LED light bulbs", "cost": 10.0},
    {"user_id": 2, "description": "rainwater harvesting system", "cost": 300.0},
    {"user_id": 2, "description": "organic vegetable seeds", "cost": 8.0},
    {"user_id": 2, "description": "sustainable food wrap (beeswax wraps)", "cost": 12.0},
    {"user_id": 2, "description": "eco-friendly cleaning products", "cost": 20.0},
    {"user_id": 2, "description": "carbon offset donation", "cost": 50.0},
    {"user_id": 1, "description": "running shoes", "cost": 150.0},
    {"user_id": 1, "description": "gym membership", "cost": 150.0},
    {"user_id": 1, "description": "protein powder", "cost": 150.0},
    {"user_id": 1, "description": "bcaa", "cost": 150.0},
    {"user_id": 1, "description": "lifting straps", "cost": 150.0},
    {"user_id": 1, "description": "shaker bottle", "cost": 150.0},
    {"user_id": 1, "description": "badminton shoes", "cost": 150.0},
    {"user_id": 2, "description": "Jay Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "Jay Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "IVE music album", "cost": 150.0},
    {"user_id": 3, "description": "IU photocards", "cost": 150.0},
    {"user_id": 3, "description": "New Jeans merchandise", "cost": 150.0},
    {"user_id": 3, "description": "Twice concert ticket", "cost": 150.0},
    {"user_id": 3, "description": "Limited edition shoes endorsed by G-dragon", "cost": 150.0},
    {"user_id": 3, "description": "Eric Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "Waterbomb music festival", "cost": 150.0},
    {"user_id": 3, "description": "Eric Chou fan sign event", "cost": 150.0},
    {"user_id": 3, "description": "Aespa photobook", "cost": 150.0}
]

# Predefined Profile Categories (same as before)
categories = [
    {"category": "Fitness guru", "description": "A Fitness Guru is someone who prioritizes physical fitness and well-being, often engaging in regular workouts, sports activities, and wellness practices. They are passionate about maintaining an active lifestyle and improving their health through exercise, nutrition, and self-care."},
    {"category": "Star Chaser", "description": "A Pop Star Chaser is someone who is deeply fascinated by celebrities, particularly pop stars, and tailors their lifestyle and spending habits around their favorite idols. They follow the latest news, social media updates, and public appearances of pop stars, attending concerts, meet-and-greet events, and exclusive fan gatherings."},
    {"category": "Politics and Society", "description": "Includes political discussions, debates, government policies, and social issues."},
    {"category": "Save the earth", "description": "A Save the Earth enthusiast's transactions reflect a lifestyle centered around environmental sustainability and conservation efforts. For instance, they might purchase a solar-powered charger to reduce reliance on non-renewable energy sources or a compost bin to promote organic waste recycling. Their shopping basket could include biodegradable cleaning products, organic cotton clothing, or sustainable fashion items, all aimed at reducing their environmental impact. They may also buy eco-friendly home appliances, such as energy-efficient refrigerators or LED lighting to lower energy consumption.In addition, this individual is likely to invest in electric vehicles or bicycles to minimize their carbon footprint, as well as reusable shopping bags and water bottles to avoid single-use plastics. They may regularly donate to environmental organizations or purchase tickets to eco-conscious events and green tech conferences. Supporting plant-based food options could also be a major part of their routine, reflecting a commitment to reducing the environmental impact of food production. They might attend beach cleanups, buy carbon offsets, or participate in tree planting activities. These purchases and actions demonstrate a consistent, conscious effort to promote sustainability and make a tangible difference in the fight against climate change."},
    {"category": "Financial freedom lo", "description": "Includes anything related to trading, stocks, options, investments, prices of gold, silver, investment products, housing, crypto."}
]

# Predefined News Genres (same as before)
news_genres = [
    {"genre": "Entertainment", "description": "Covers movies, music, celebrity news, TV shows, and all things related to entertainment."},
    {"genre": "Politics", "description": "In-depth coverage of political news, elections, government policies, and political events."},
    {"genre": "Environment", "description": "overs topics related to the natural world, climate change, sustainability, and environmental conservation efforts. It highlights the latest developments in global and local environmental issues, such as the impact of human activities on ecosystems, wildlife conservation, deforestation, pollution, and the depletion of natural resources. Environmental news also focuses on efforts to combat climate change, including renewable energy initiatives, conservation programs, and governmental policies aimed at reducing carbon emissions. This genre covers cutting-edge research on alternative energy, environmental activism, climate policy, as well as community-driven sustainability efforts. It often includes updates on eco-friendly technologies, environmental regulations, and corporate efforts to go green. With the rising awareness of climate change and environmental degradation, this genre plays a crucial role in informing the public about how actions today can affect the planet’s future and what steps are being taken to protect it."},
    {"genre": "Health & Fitness", "description": "Focuses on the latest developments in exercise, nutrition, wellness trends, and overall well-being."},
    {"genre": "Technology", "description": "Covers tech news, gadgets, innovations, software, and the world of technology."}
]

# Sample News Articles (you can add more)
news_articles = [
    {"message": "New studies show the increasing effects of climate change on coastal cities.", "category": "Environment"},
    {"message": "Renewable energy technologies are advancing rapidly and promise a greener future.", "category": "Environment"},
    {"message": "Hollywood stars gather to discuss sustainable fashion at global event.", "category": "Entertainment"},
    {"message": "Political leaders debate new laws to protect the environment from corporate polluters.", "category": "Politics"},
    {"message": "The latest advancements in health and fitness technology are revolutionizing the industry.", "category": "Health & Fitness"},
    {"message": "Tech companies announce new AI-driven tools for environmental protection.", "category": "Technology"}
]

# Elasticsearch Indexing Functions
# Create News Article Index
def create_news_article_index():
    news_article_mapping = {
        "mappings": {
            "properties": {
                "message": {"type": "text"},
                "category": {"type": "keyword"},
                "article_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="news_articles"):
        es.indices.create(index="news_articles", body=news_article_mapping)
        print("News articles index created.")

def create_transaction_index():
    transaction_mapping = {
        "mappings": {
            "properties": {
                "user_id": {"type": "keyword"},
                "description": {"type": "text"},
                "cost": {"type": "float"},
                "transaction_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_transactions"):
        es.indices.create(index="user_transactions", body=transaction_mapping)
        print("Transactions index created.")

def create_category_index():
    category_mapping = {
        "mappings": {
            "properties": {
                "category": {"type": "keyword"},
                "category_description": {"type": "text"},
                "category_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_categories"):
        es.indices.create(index="user_categories", body=category_mapping)
        print("Category index created.")

def create_news_genre_index():
    news_genre_mapping = {
        "mappings": {
            "properties": {
                "genre": {"type": "keyword"},
                "genre_description": {"type": "text"},
                "genre_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_news_genres"):
        es.indices.create(index="user_news_genres", body=news_genre_mapping)
        print("News genres index created.")

# Generate Embeddings for Transaction Descriptions
def generate_transaction_embedding(description):
    return model.encode(description)

# Generate Embeddings for Category Descriptions
def generate_category_description_embedding(description):
    return model.encode(description)

# Generate Embeddings for News Genres Descriptions
def generate_news_genre_embedding(description):
    return model.encode(description)

# Index Sample Transactions
def index_transactions():
    for transaction in transactions:
        description = transaction["description"]
        user_id = transaction["user_id"]  # Extract user_id
        embedding = generate_transaction_embedding(description)

        es.index(index="user_transactions", document={
            "user_id": user_id,  # Include user_id in the document
            "description": description,
            "cost": transaction["cost"],
            "transaction_vector": embedding.tolist()
        })
    print("Transactions indexed.")

# Index Profile Categories with Descriptions
def index_categories():
    for category in categories:
        embedding = generate_category_description_embedding(category["description"])

        es.index(index="user_categories", document={
            "category": category["category"],
            "category_description": category["description"],
            "category_vector": embedding.tolist()
        })
    print("Profile categories indexed.")

# Index News Genres with Descriptions
def index_news_genres():
    for genre in news_genres:
        embedding = generate_news_genre_embedding(genre["description"])

        es.index(index="user_news_genres", document={
            "genre": genre["genre"],
            "genre_description": genre["description"],
            "genre_vector": embedding.tolist()
        })
    print("News genres indexed.")

# Index Sample News Articles
def index_news_articles():
    for article in news_articles:
        message = article["message"]
        category = article["category"]
        embedding = model.encode(message)  # Generate embedding for the article message

        es.index(index="news_articles", document={
            "message": message,
            "category": category,
            "article_vector": embedding.tolist()
        })
    print("News articles indexed.")

# Step 1: Aggregate a User's Transactions
def aggregate_user_transactions(user_id):
    # Filter transactions by user_id
    user_transactions = [t["description"] for t in transactions if t["user_id"] == user_id]

    # Combine all transaction descriptions for the user into one string
    aggregated_description = " ".join(user_transactions)

    return aggregated_description

# Step 2: Find the most relevant category based on aggregated transaction descriptions
def find_relevant_category(user_id):
    aggregated_description = aggregate_user_transactions(user_id)

    # Generate embedding for aggregated description
    user_embedding = generate_transaction_embedding(aggregated_description)

    # Query Elasticsearch for categories and calculate cosine similarity with category embeddings
    query = {
        "size": 1000,  # Fetch all categories
        "_source": ["category", "category_description", "category_vector"]
    }
    response = es.search(index="user_categories", body=query)

    category_embeddings = []
    for hit in response["hits"]["hits"]:
        category = hit["_source"]["category"]
        category_description = hit["_source"]["category_description"]
        embedding = np.array(hit["_source"]["category_vector"])
        category_embeddings.append((category, category_description, embedding))

    # Find the most relevant category based on cosine similarity
    similarities = []
    for category, category_description, embedding in category_embeddings:
        similarity = cosine_similarity([user_embedding], [embedding])[0][0]
        similarities.append((category, similarity))

    # Return the category with the highest similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_category = similarities[0][0]

    return most_relevant_category

# Step 3: Find the most relevant news genre based on the selected category's embedding
def find_relevant_news_genre(category):
    category_description = next(c['description'] for c in categories if c['category'] == category)
    category_embedding = generate_category_description_embedding(category_description)

    query = {
        "size": 1000,  # Fetch all genres
        "_source": ["genre", "genre_description", "genre_vector"]
    }
    response = es.search(index="user_news_genres", body=query)

    genre_embeddings = []
    for hit in response["hits"]["hits"]:
        genre = hit["_source"]["genre"]
        genre_description = hit["_source"]["genre_description"]
        embedding = np.array(hit["_source"]["genre_vector"])
        genre_embeddings.append((genre, genre_description, embedding))

    similarities = []
    for genre, genre_description, embedding in genre_embeddings:
        similarity = cosine_similarity([category_embedding], [embedding])[0][0]
        similarities.append((genre, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_genre = similarities[0][0]

    return most_relevant_genre
# Step 4: Fetch News Articles Based on Genre
def find_news_articles_by_genre(genre):
    # Query Elasticsearch for articles in the given genre
    query = {
        "size": 1000,  # Fetch all articles
        "_source": ["message", "category", "article_vector"],
        "query": {
            "match": {
                "category": genre
            }
        }
    }
    response = es.search(index="news_articles", body=query)

    # Collect matching articles
    articles = []
    for hit in response["hits"]["hits"]:
        message = hit["_source"]["message"]
        category = hit["_source"]["category"]
        articles.append({"message": message, "category": category})

    return articles

# Main function to run the workflow
def main(user_id):
    create_transaction_index()
    create_category_index()
    create_news_genre_index()
    create_news_article_index()

    # Index sample transactions, categories, news genres, and news articles
    index_transactions()
    index_categories()
    index_news_genres()
    index_news_articles()

    # Find the relevant category and news genre for the specified user
    relevant_category = find_relevant_category(user_id)
    print(f"Relevant Category for User {user_id}: {relevant_category}")

    relevant_genre = find_relevant_news_genre(relevant_category)
    print(f"Relevant News Genre for User {user_id}: {relevant_genre}")

    # Fetch and return the news articles related to the recommended genre
    news_articles = find_news_articles_by_genre(relevant_genre)
    print(f"News Articles for Genre '{relevant_genre}':")
    for article in news_articles:
        print(f" - {article['message']}")

# Run the main function with user_id = 1 as an example
if __name__ == "__main__":
    main(user_id=2)


Transactions indexed.
Profile categories indexed.
News genres indexed.
Relevant Category for User 2: Save the earth
Relevant News Genre for User 2: Environment


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

# Initialize Elasticsearch and the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample Transactions (assuming it's in the form of a list of dictionaries)
transactions = [
    {"user_id": 2, "description": "reusable water bottle", "cost": 20.0},
    {"user_id": 2, "description": "organic cotton t-shirt", "cost": 25.0},
    {"user_id": 2, "description": "solar-powered charger", "cost": 50.0},
    {"user_id": 2, "description": "bamboo toothbrush", "cost": 5.0},
    {"user_id": 2, "description": "composting bin", "cost": 40.0},
    {"user_id": 2, "description": "eco-friendly laundry detergent", "cost": 15.0},
    {"user_id": 2, "description": "recycled paper notebooks", "cost": 10.0},
    {"user_id": 2, "description": "plant-based protein powder", "cost": 30.0},
    {"user_id": 2, "description": "electric car charging station subscription", "cost": 100.0},
    {"user_id": 2, "description": "LED light bulbs", "cost": 10.0},
    {"user_id": 2, "description": "rainwater harvesting system", "cost": 300.0},
    {"user_id": 2, "description": "organic vegetable seeds", "cost": 8.0},
    {"user_id": 2, "description": "sustainable food wrap (beeswax wraps)", "cost": 12.0},
    {"user_id": 2, "description": "eco-friendly cleaning products", "cost": 20.0},
    {"user_id": 2, "description": "carbon offset donation", "cost": 50.0},
    {"user_id": 1, "description": "running shoes", "cost": 150.0},
    {"user_id": 1, "description": "gym membership", "cost": 150.0},
    {"user_id": 1, "description": "protein powder", "cost": 150.0},
    {"user_id": 1, "description": "bcaa", "cost": 150.0},
    {"user_id": 1, "description": "lifting straps", "cost": 150.0},
    {"user_id": 1, "description": "shaker bottle", "cost": 150.0},
    {"user_id": 1, "description": "badminton shoes", "cost": 150.0},
    {"user_id": 3, "description": "Jay Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "Eric Chou concert", "cost": 150.0},
    {"user_id": 3, "description": "IVE music album", "cost": 150.0},
    {"user_id": 3, "description": "IU photocards", "cost": 150.0},
    {"user_id": 3, "description": "IU merchandise", "cost": 150.0},
    {"user_id": 3, "description": "Twice concert ticket", "cost": 150.0},
    {"user_id": 3, "description": "Limited edition shoes endorsed by G-dragon", "cost": 150.0},
    {"user_id": 3, "description": "Puma x Some pop star collab", "cost": 150.0},
    {"user_id": 3, "description": "Gentle monster endorsed by Jennie", "cost": 150.0},
]

# Predefined Profile Categories (same as before)
categories = [
    {"category": "Health Conscious", "description": "A Fitness Guru is someone who prioritizes physical fitness and well-being, often engaging in regular workouts, sports activities, and wellness practices. They are passionate about maintaining an active lifestyle and improving their health through exercise, nutrition, and self-care."},
    {"category": "Star Chaser", "description": "A Pop Star Chaser is someone who is deeply fascinated by celebrities, particularly pop stars, and tailors their lifestyle and spending habits around their favorite idols. They follow the latest news, social media updates, and public appearances of pop stars, attending concerts, meet-and-greet events, and exclusive fan gatherings."},
    {"category": "Political", "description": "Includes political discussions, debates, government policies, and social issues."},
    {"category": "Save the earth", "description": "A Save the Earth enthusiast's transactions reflect a lifestyle centered around environmental sustainability and conservation efforts. For instance, they might purchase a solar-powered charger to reduce reliance on non-renewable energy sources or a compost bin to promote organic waste recycling. Their shopping basket could include biodegradable cleaning products, organic cotton clothing, or sustainable fashion items, all aimed at reducing their environmental impact. They may also buy eco-friendly home appliances, such as energy-efficient refrigerators or LED lighting to lower energy consumption.In addition, this individual is likely to invest in electric vehicles or bicycles to minimize their carbon footprint, as well as reusable shopping bags and water bottles to avoid single-use plastics. They may regularly donate to environmental organizations or purchase tickets to eco-conscious events and green tech conferences. Supporting plant-based food options could also be a major part of their routine, reflecting a commitment to reducing the environmental impact of food production. They might attend beach cleanups, buy carbon offsets, or participate in tree planting activities. These purchases and actions demonstrate a consistent, conscious effort to promote sustainability and make a tangible difference in the fight against climate change."},
    {"category": "Judgemental", "description": "Includes anything related to trading, stocks, options, investments, prices of gold, silver, investment products, housing, crypto."}
]

# Predefined News Genres (same as before)
news_genres = [
    {"genre": "Entertainment", "description": "Covers movies, music, celebrity news, TV shows, and all things related to entertainment."},
    {"genre": "Politics", "description": "In-depth coverage of political news, elections, government policies, and political events."},
    {"genre": "Environment", "description": "overs topics related to the natural world, climate change, sustainability, and environmental conservation efforts. It highlights the latest developments in global and local environmental issues, such as the impact of human activities on ecosystems, wildlife conservation, deforestation, pollution, and the depletion of natural resources. Environmental news also focuses on efforts to combat climate change, including renewable energy initiatives, conservation programs, and governmental policies aimed at reducing carbon emissions. This genre covers cutting-edge research on alternative energy, environmental activism, climate policy, as well as community-driven sustainability efforts. It often includes updates on eco-friendly technologies, environmental regulations, and corporate efforts to go green. With the rising awareness of climate change and environmental degradation, this genre plays a crucial role in informing the public about how actions today can affect the planet’s future and what steps are being taken to protect it."},
    {"genre": "Health & Fitness", "description": "Focuses on the latest developments in exercise, nutrition, wellness trends, and overall well-being."},
    {"genre": "Technology", "description": "Covers tech news, gadgets, innovations, software, and the world of technology."}
]

# Sample News Articles (you can add more)
news_articles = [
    {"message": "New studies show the increasing effects of climate change on coastal cities.", "category": "Environment"},
    {"message": "Renewable energy technologies are advancing rapidly and promise a greener future.", "category": "Environment"},
    {"message": "Hollywood stars gather to discuss sustainable fashion at global event.", "category": "Entertainment"},
    {"message": "Political leaders debate new laws to protect the environment from corporate polluters.", "category": "Politics"},
    {"message": "The latest advancements in health and fitness technology are revolutionizing the industry.", "category": "Health & Fitness"},
    {"message": "Tech companies announce new AI-driven tools for environmental protection.", "category": "Technology"}
]

# Elasticsearch Indexing Functions
# Create News Article Index
def create_news_article_index():
    news_article_mapping = {
        "mappings": {
            "properties": {
                "message": {"type": "text"},
                "category": {"type": "keyword"},
                "article_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="news_articles"):
        es.indices.create(index="news_articles", body=news_article_mapping)
        print("News articles index created.")

def create_transaction_index():
    transaction_mapping = {
        "mappings": {
            "properties": {
                "user_id": {"type": "keyword"},
                "description": {"type": "text"},
                "cost": {"type": "float"},
                "transaction_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_transactions"):
        es.indices.create(index="user_transactions", body=transaction_mapping)
        print("Transactions index created.")

def create_category_index():
    category_mapping = {
        "mappings": {
            "properties": {
                "category": {"type": "keyword"},
                "category_description": {"type": "text"},
                "category_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_categories"):
        es.indices.create(index="user_categories", body=category_mapping)
        print("Category index created.")

def create_news_genre_index():
    news_genre_mapping = {
        "mappings": {
            "properties": {
                "genre": {"type": "keyword"},
                "genre_description": {"type": "text"},
                "genre_vector": {
                    "type": "dense_vector",
                    "dims": 384  # Sentence Transformer embedding dimensionality
                }
            }
        }
    }

    if not es.indices.exists(index="user_news_genres"):
        es.indices.create(index="user_news_genres", body=news_genre_mapping)
        print("News genres index created.")

# Generate Embeddings for Transaction Descriptions
def generate_transaction_embedding(description):
    return model.encode(description)

# Generate Embeddings for Category Descriptions
def generate_category_description_embedding(description):
    return model.encode(description)

# Generate Embeddings for News Genres Descriptions
def generate_news_genre_embedding(description):
    return model.encode(description)

# Index Sample Transactions
def index_transactions():
    for transaction in transactions:
        description = transaction["description"]
        user_id = transaction["user_id"]  # Extract user_id
        embedding = generate_transaction_embedding(description)

        es.index(index="user_transactions", document={
            "user_id": user_id,  # Include user_id in the document
            "description": description,
            "cost": transaction["cost"],
            "transaction_vector": embedding.tolist()
        })
    print("Transactions indexed.")

# Index Profile Categories with Descriptions
def index_categories():
    for category in categories:
        embedding = generate_category_description_embedding(category["description"])

        es.index(index="user_categories", document={
            "category": category["category"],
            "category_description": category["description"],
            "category_vector": embedding.tolist()
        })
    print("Profile categories indexed.")

# Index News Genres with Descriptions
def index_news_genres():
    for genre in news_genres:
        embedding = generate_news_genre_embedding(genre["description"])

        es.index(index="user_news_genres", document={
            "genre": genre["genre"],
            "genre_description": genre["description"],
            "genre_vector": embedding.tolist()
        })
    print("News genres indexed.")

# Index Sample News Articles
def index_news_articles():
    for article in news_articles:
        message = article["message"]
        category = article["category"]
        embedding = model.encode(message)  # Generate embedding for the article message

        es.index(index="news_articles", document={
            "message": message,
            "category": category,
            "article_vector": embedding.tolist()
        })
    print("News articles indexed.")

# Step 1: Aggregate a User's Transactions
def aggregate_user_transactions(user_id):
    # Filter transactions by user_id
    user_transactions = [t["description"] for t in transactions if t["user_id"] == user_id]

    # Combine all transaction descriptions for the user into one string
    aggregated_description = " ".join(user_transactions)

    return aggregated_description

# Step 2: Find the most relevant category based on aggregated transaction descriptions
def find_relevant_category(user_id):
    aggregated_description = aggregate_user_transactions(user_id)

    # Generate embedding for aggregated description
    user_embedding = generate_transaction_embedding(aggregated_description)

    # Query Elasticsearch for categories and calculate cosine similarity with category embeddings
    query = {
        "size": 1000,  # Fetch all categories
        "_source": ["category", "category_description", "category_vector"]
    }
    response = es.search(index="user_categories", body=query)

    category_embeddings = []
    for hit in response["hits"]["hits"]:
        category = hit["_source"]["category"]
        category_description = hit["_source"]["category_description"]
        embedding = np.array(hit["_source"]["category_vector"])
        category_embeddings.append((category, category_description, embedding))

    # Find the most relevant category based on cosine similarity
    similarities = []
    for category, category_description, embedding in category_embeddings:
        similarity = cosine_similarity([user_embedding], [embedding])[0][0]
        similarities.append((category, similarity))

    # Return the category with the highest similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_category = similarities[0][0]

    return most_relevant_category

# Step 3: Find the most relevant news genre based on the selected category's embedding
def find_relevant_news_genre(category):
    category_description = next(c['description'] for c in categories if c['category'] == category)
    category_embedding = generate_category_description_embedding(category_description)

    query = {
        "size": 1000,  # Fetch all genres
        "_source": ["genre", "genre_description", "genre_vector"]
    }
    response = es.search(index="user_news_genres", body=query)

    genre_embeddings = []
    for hit in response["hits"]["hits"]:
        genre = hit["_source"]["genre"]
        genre_description = hit["_source"]["genre_description"]
        embedding = np.array(hit["_source"]["genre_vector"])
        genre_embeddings.append((genre, genre_description, embedding))

    similarities = []
    for genre, genre_description, embedding in genre_embeddings:
        similarity = cosine_similarity([category_embedding], [embedding])[0][0]
        similarities.append((genre, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    most_relevant_genre = similarities[0][0]

    return most_relevant_genre
# Step 4: Fetch News Articles Based on Genre
def find_news_articles_by_genre(genre):
    # Query Elasticsearch for articles in the given genre
    query = {
        "size": 1000,  # Fetch all articles
        "_source": ["message", "category", "article_vector"],
        "query": {
            "match": {
                "category": genre
            }
        }
    }
    response = es.search(index="news_articles", body=query)

    # Collect matching articles
    articles = []
    for hit in response["hits"]["hits"]:
        message = hit["_source"]["message"]
        category = hit["_source"]["category"]
        articles.append({"message": message, "category": category})

    return articles

# Main function to run the workflow
def main(user_id):
    create_transaction_index()
    create_category_index()
    create_news_genre_index()
    create_news_article_index()

    # Index sample transactions, categories, news genres, and news articles
   # index_transactions()
   # index_categories()
   # index_news_genres()
   # index_news_articles()

    # Find the relevant category and news genre for the specified user
    relevant_category = find_relevant_category(user_id)
    print(f"Relevant Category for User {user_id}: {relevant_category}")

    relevant_genre = find_relevant_news_genre(relevant_category)
    print(f"Relevant News Genre for User {user_id}: {relevant_genre}")

    # Fetch and return the news articles related to the recommended genre
    news_articles = find_news_articles_by_genre(relevant_genre)
    print(f"News Articles for Genre '{relevant_genre}':")
    for article in news_articles:
        print(f" - {article['message']}")

# Run the main function with user_id = 1 as an example
if __name__ == "__main__":
    main(user_id=2)


Relevant Category for User 2: Save the earth
Relevant News Genre for User 2: Environment
News Articles for Genre 'Environment':
 - New studies show the increasing effects of climate change on coastal cities.
 - New studies show the increasing effects of climate change on coastal cities.
 - Renewable energy technologies are advancing rapidly and promise a greener future.
 - Renewable energy technologies are advancing rapidly and promise a greener future.
